# Capacity Credit Calculator

## **Usage**

1. In Google Colab, press `ctrl+F9`
1. Press `run anyway` if a warning message appear.
1. Wait for the UI to show up (until a loading circle in the bottom of notebook stopped).
1. Upload files by pressing upload button.
1. Press compute to run simulation.
1. Press `ctrl+F7` to restart.

## **Input Format**

### Forced outage rate input format

|No |Unit Name  |Capacity (MW)  |FOR (Outage)   |Status |
|---|-----------|---------------|---------------|-------|
|1  |COAL-1     |50             |0.1            |1      |
|2  |COAL-2     |20             |0.1            |0      |
|3  |COAL-3     |50             |0.2            |1      |
|4  |COAL-4     |30             |0.1            |1      |

Note:

1. status denotes that the generating unit is considered (1) or not (0) in making COPT table.

### Profile input format

|No     |Demand (MW)    |
|-------|---------------|
|1      |150            |
|...    |...            |
|...    |...            |
|8760   |125            |

### VOLL input format

VOLL can use `_` and `,` separator for thousands and `.` for cents, eg: `15_000_000.00`


In [1]:
# @title
import itertools

import ipywidgets
import numpy as np
import pandas as pd
from ipywidgets import widgets

COPT_COLUMNS = ['Combined Capacity',
                'Individual Probability',
                'Cumulative Probability',
                'Reversed Cumulative Probability']

# setting numpy print option decimal places
decimal_places = 4
np.set_printoptions(precision=decimal_places, suppress=True)
pd.set_option('display.float_format', '{:.4f}'.format)


# copt
def get_copt(capacities, outage_rates, status):
    # filter only available generator
    generator_list = [[cap, 1 - out]
                      for cap, out, stat
                      in sorted(zip(capacities, outage_rates, status), reverse=True)
                      if stat]

    # make tables for each generator
    # TODO: Tables already support derating, input data should also support it
    # from excel
    tables = [np.array([generator, [0, 1 - generator[1]]])
              for generator in generator_list]

    table = tables[0].copy()  # copy to avoid modifying data
    for table_ in tables[1:]:
        # faster than flatten + transpose
        table = np.hstack(((table[:, 0, None] + table_[:, 0])  # sum capacity
                           .reshape(-1, 1),
                           (table[:, 1, None] * table_[:, 1])  # multiply probability
                           .reshape(-1, 1),
                           ))

        # sort table
        table = table[(-table[:, 0]).argsort(), :]

        # combine duplicate
        table = np.array([[k, sum([x[1] for x in list(g)])]
                          for k, g in itertools.groupby(table, lambda x:x[0])])

        # TODO: 
        # Implement resample,
        # useful for big COPT table, triggered only if max table length achieve

    table = np.hstack((table,
                       # Cumulative Probability
                       np.atleast_2d(np.cumsum(table[::-1, 1])[::-1]).T,
                       # Reversed Cumulative Probability
                       np.atleast_2d(np.cumsum(table[:, 1])).T,
                       ))

    return table


# lolp
def get_lolp(capacity, cumulative_probability, demand):
    """
    format:
        capacity (descend)
        cumulative_probability(descend)
    """
    try:
        idx = np.where(capacity < demand)[0][0]
    except IndexError:
        idx = -1
    return cumulative_probability[idx]


# eens
def get_eens(capacity, individual_probability, demand):
    # TODO: standardize input in the form of one-year data (8760 data) by default
    """
    format:
        capacity
        individual_probability
    """
    return sum(individual_probability
               * (capacity < demand)
               * (demand - capacity))


def find_delta_load(lb, ub, table, net_loads, eens, tol=0.0001):
    # lb_0 = 0
    # ub_0 = max(sun)
    # delta_load_0 = max(sun) / 2

    eens_delta_load = np.inf
    while abs(eens_delta_load - eens) > tol:
        delta_load = (lb + ub) * 0.5
        eens_delta_load = sum([get_eens(table[:, 0], table[:, 1], net_load)
                               for net_load in net_loads + delta_load])
        if eens_delta_load > eens:
            ub = delta_load
        else:
            lb = delta_load
    return delta_load, eens_delta_load

In [2]:
# @title
# Uploader class for input
class Uploader:
    def __init__(self, name=None):
        if name is None:
            self._name = ' '
        else:
            self._name = f' {name} '

        # uploader
        self._file_upload = widgets.FileUpload(
            accept='.csv*',  # Accepted file extension
            multiple=False  # True to accept multiple files upload else False
        )

        self.box = widgets.VBox(
            children=(
                self._file_upload,
                widgets.Label(value=f'Upload{self._name}in .csv Format')
                )
            )

    def get_dataframe(self):
        return get_dataframe_from_widget(self._file_upload)


# widgets
def write_file_from_bytes_data(bytes_data, file_name='NAME.csv'):
    with open(file_name, 'wb') as f:
        f.write(bytes_data)


def _get_content_and_name_ipywidget_8(file_upload):
    return (
        file_upload.value[0]['content'],
        file_upload.value[0]['name'],
    )


def _get_content_and_name_ipywidget_7(file_upload):
    for _key, value in file_upload.value.items():
        return (
            value['content'],
            value['metadata']['name']
        )

# get ipywidgets version
IPYWIDGETS_VERSION = ipywidgets.__version__.split('.')[0]
if IPYWIDGETS_VERSION == '8':
    get_content_and_name = _get_content_and_name_ipywidget_8
elif IPYWIDGETS_VERSION == '7':
    get_content_and_name = _get_content_and_name_ipywidget_7
else:
    print('Please install either ipywidgets 7 or 8, for example:')
    print('pip install ipywidgets==7.7.2')


def get_dataframe_from_widget(file_upload):
    # get content and file name
    content, name = get_content_and_name(file_upload)

    # from bytes data to csv file
    write_file_from_bytes_data(content, file_name=name)

    # pandas read csv
    return pd.read_csv(name)

In [3]:
# @title
# initiate widgets
output = widgets.Output()
for_data_uploader = Uploader(name='FOR Data')
demand_profile_data_uploader = Uploader(name='Demand Profile Data')
vre_profile_data_uploadder = Uploader(name='VRE Profile Data')
voll_widget = widgets.Text(
    value='15_000_000.00',
    description='VOLL: ',
    disabled=False
)
currency_widget = widgets.Dropdown(
    options=['IDR', 'USD'],
    value='IDR',
    description='Currency: ',
    disabled=False,
)
button = widgets.Button(description="Compute!")

In [4]:
# @title
# button action
def on_button_clicked(b):
    with output:
        # filter for inputs
        df = for_data_uploader.get_dataframe()
        capacities = df['Capacity (MW)'].tolist()
        outage_rates = df['FOR (Outage)'].tolist()
        status = df['Status'].tolist()

        # # to print input table, use:
        # pd.DataFrame(data={'capacities': capacities,
        #                    'outage_rates': outage_rates,
        #                    'status': status},
        #              index=pd.RangeIndex(1, len(capacities) + 1, 1)).head(10)

        table = get_copt(capacities, outage_rates, status)
        df = pd.DataFrame(data=table,
                          columns=COPT_COLUMNS,
                          index=pd.RangeIndex(1, len(table) + 1, 1, name='No'),
                          )
        df.head(10)  # to print copt table
        # to save COPT in csv
        # CASE_NAME = 'COPT Case.csv'
        # df.to_csv(f'../results/COPT_{CASE_NAME}')

        # TODO: copt downloader

        # filter demand inputs
        demands = demand_profile_data_uploader.get_dataframe()['Demand (MW)'].values
        eens = sum([get_eens(table[:, 0], table[:, 1], demand)
                    for demand in demands])
        print(f'Total EENS (using demand profile): {eens:.4f} MW')

        vres = vre_profile_data_uploadder.get_dataframe().set_index('No')
        vres_cf = vres['Capacity Factor (p.u.)'].values
        for column in vres.columns[1:]:
            print(f'Using demand and {column} scenario:')

            vres_max_cap = vres[column].values[0]
            profile = vres_cf * vres_max_cap
            net_loads = demands - profile
            print(f'    Installed capacity: {vres_max_cap} MWp')

            # EENS
            eens = sum([get_eens(table[:, 0], table[:, 1], net_load)
                        for net_load in net_loads])
            
            print(f'    Total EENS: {eens:.4f} MW')

            delta_load, eens_delta_load = find_delta_load(
                0, np.max(profile), table, net_loads, eens, tol=0.01)
            print(f'    Equal to flat load: {delta_load:.4f} MW')
            print(f'    Capacity factor: {100 * (delta_load / vres_max_cap):.4f} %')


# initiate button
button.on_click(on_button_clicked)

In [5]:
# @title
# interface
widgets.VBox((for_data_uploader.box,
              demand_profile_data_uploader.box,
              vre_profile_data_uploadder.box,
              button,
              output,
              ))